In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import xml.etree.ElementTree as ET
import time

from requests.auth import HTTPBasicAuth

In [16]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

# Request access token
response = requests.post(
    token_url,
    auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
    data={"grant_type": "client_credentials"},
)

# Extract token
if response.status_code == 200:
    token = response.json()["access_token"]
    print("Access Token:", token)
else:
    print("Failed to get token:", response.text)

Access Token: B7k91V5ahRKG3qDbSU43A5ZzFscm


In [23]:
def search_for_patents_by_year(year, token, b = 1901, e = 2000):
    search_url = "https://ops.epo.org/rest-services/published-data/search"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
    }

    # Parameters: Retrieve all patents (max 100 at a time)
    params = {
        "q": f'pd within "{year},{year}"',
        "Range": f"{b}-{e}"
    }
    
    # Send request
    response = requests.get(search_url, headers=headers, params=params)

    # Check response
    if response.status_code == 200:
        xml_data = response.text
        return xml_data
    else:
        return 
        

print(search_for_patents_by_year(1883, token))

<?xml version="1.0" encoding="UTF-8"?><?xml-stylesheet type="text/xsl" href="../../style/exchange.xsl"?>
<ops:world-patent-data xmlns="http://www.epo.org/exchange" xmlns:ops="http://ops.epo.org" xmlns:xlink="http://www.w3.org/1999/xlink">
    <ops:biblio-search total-result-count="10000">
        <ops:query syntax="CQL">pd within "1883,1883"</ops:query>
        <ops:range begin="1901" end="2000"/>
        <ops:search-result>
            <ops:publication-reference system="ops.epo.org" family-id="45134017">
                <document-id document-id-type="docdb">
                    <country>CA</country>
                    <doc-number>17474</doc-number>
                    <kind>A</kind>
                </document-id>
            </ops:publication-reference>
            <ops:publication-reference system="ops.epo.org" family-id="45134016">
                <document-id document-id-type="docdb">
                    <country>CA</country>
                    <doc-number>17473</doc-number>
    

In [5]:
def extract_patent_data(xml_data):
    
    # Check if xml_data is valid
    if not xml_data:
        #print("No data to parse.")
        return []
    
    # Parse the XML data
    try:
        tree = ET.ElementTree(ET.fromstring(xml_data))
        root = tree.getroot()
    except ET.ParseError:
        print("Error parsing the XML data.")
        return []

    namespaces = {'': 'http://www.epo.org/exchange'}  # Default namespace
    
    patent_data = []  # List to store the extracted data

    # Loop through each publication reference and extract the country and doc-number
    for publication in root.findall(".//ops:publication-reference", namespaces={"ops": "http://ops.epo.org"}):
        # Get the country from each publication reference
        country = publication.find(".//document-id/country", namespaces=namespaces)
        doc_number = publication.find(".//document-id/doc-number", namespaces=namespaces)

        # Collect data if available
        if country is not None and doc_number is not None:
            patent_data.append({
                "country": country.text,
                "doc_number": doc_number.text
            })

    return patent_data


xml_data = search_for_patents_by_year(1883, token)  # Get XML data for a specific year
patent_info = extract_patent_data(xml_data)  # Extract country and doc-number

# Output the result for verification
print(patent_info)
print(len(patent_info))

[{'country': 'US', 'doc_number': '289418'}, {'country': 'ES', 'doc_number': '3379'}, {'country': 'ES', 'doc_number': '3463'}, {'country': 'ES', 'doc_number': '3364'}, {'country': 'ES', 'doc_number': '3336'}, {'country': 'ES', 'doc_number': '3298'}, {'country': 'ES', 'doc_number': '3123'}, {'country': 'ES', 'doc_number': '3071'}, {'country': 'ES', 'doc_number': '3062'}, {'country': 'ES', 'doc_number': '3082'}, {'country': 'ES', 'doc_number': '2841'}, {'country': 'ES', 'doc_number': '3023'}, {'country': 'ES', 'doc_number': '2923'}, {'country': 'ES', 'doc_number': '2818'}, {'country': 'ES', 'doc_number': '2761'}, {'country': 'ES', 'doc_number': '2650'}, {'country': 'ES', 'doc_number': '2491'}, {'country': 'ES', 'doc_number': '2510'}, {'country': 'ES', 'doc_number': '2411'}, {'country': 'ES', 'doc_number': '2840'}, {'country': 'ES', 'doc_number': '2793'}, {'country': 'ES', 'doc_number': '3652'}, {'country': 'ES', 'doc_number': '3606'}, {'country': 'ES', 'doc_number': '3451'}, {'country': '

In [6]:
def patent_names(lookup):
    
    doc_number = lookup["country"] + "" + lookup["doc_number"] #
    
    search_url = f"https://ops.epo.org/rest-services/published-data/publication/epodoc/{doc_number}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
    }
    
    response = requests.get(search_url, headers=headers)
    
    # Parse XML recieved by the requested patent
    root = ET.fromstring(response.text)
    
    # Define namespace to search the patent properly
    ns = {"epo": "http://www.epo.org/exchange"}

    # Extract title
    title_element = root.find(".//epo:invention-title", ns)
    title = title_element.text if title_element is not None else "Unknown Title"

    # Try to extract inventor name first
    inventor_element = root.find(".//epo:inventor-name/epo:name", ns)
    if inventor_element is not None:
        inventor = " ".join(inventor_element.text.split()).strip()
    else:
        # If inventor not found, extract applicant name
        #applicant_element = root.find(".//epo:applicants//epo:applicant-name", ns)
        applicant_element = root.find(".//epo:applicants/epo:applicant/epo:applicant-name/epo:name", ns)

        inventor = "Unknown Applicant"
        if applicant_element is not None:
            inventor = " ".join(applicant_element.text.split()).strip()

    return {"title": title, "inventor": inventor}


    

    
patent_names(patent_info[1])

{'title': 'Perfeccionamientos a las máquinas dinamo eléctricas.',
 'inventor': 'DALMAU TOMAS JOSE'}

In [32]:
patent_test = patent_info[:3]

def get_patent_names_and_inventor_names(patent_subset):
    
    for patent in patent_subset:
        print(patent_names(patent))
        
get_patent_names_and_inventor_names(patent_test)

{'title': 'TREADLE', 'inventor': 'JOHNSON WILLIAM OSCAR [US]'}
{'title': 'Perfeccionamientos a las máquinas dinamo eléctricas.', 'inventor': 'DALMAU TOMAS JOSE'}
{'title': 'Perfeccionamientos introducidos en los motores de gas.', 'inventor': 'BONNIN EUGENIO'}


In [ ]:
all_patents = []

for year in range(1700,2026):
    xml_data = search_for_patents_by_year(year, token)
    #print(xml_data)
    patent_info = extract_patent_data(xml_data)
    
    
    
    time.sleep(1)
        


No patents found in year 1700
No patents found in year 1701
No patents found in year 1702
No patents found in year 1703
No patents found in year 1704
No patents found in year 1705
No patents found in year 1706
No patents found in year 1707
No patents found in year 1708
No patents found in year 1709
No patents found in year 1710
No patents found in year 1711
No patents found in year 1712
No patents found in year 1713
No patents found in year 1714
No patents found in year 1715
No patents found in year 1716
No patents found in year 1717
No patents found in year 1718
No patents found in year 1719
No patents found in year 1720
No patents found in year 1721
No patents found in year 1722
No patents found in year 1723
No patents found in year 1724
No patents found in year 1725
No patents found in year 1726
No patents found in year 1727
No patents found in year 1728
No patents found in year 1729
No patents found in year 1730
No patents found in year 1731
No patents found in year 1732
No patents

In [23]:
print(len(all_patents))

326


In [24]:
csv_filename = "patents.csv"

with open(csv_filename, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    
    # Write header
    writer.writerow(["year", "country", "doc_number"])
    
    # Flatten the data while writing
    for year_entry in all_patents:
        year = year_entry[0]
        patent_list = year_entry[1]

        for patent in patent_list:
            writer.writerow([year, patent["country"], patent["doc_number"]])

print(f"CSV file '{csv_filename}' has been created successfully.")

CSV file 'patents.csv' has been created successfully.


In [11]:
def new_extract_patent_data(xml_data):
    
    # Check if xml_data is valid
    if not xml_data:
        #print("No data to parse.")
        return []
    
    # Parse the XML data
    try:
        tree = ET.ElementTree(ET.fromstring(xml_data))
        root = tree.getroot()
    except ET.ParseError:
        print("Error parsing the XML data.")
        return []

    namespaces = {'': 'http://www.epo.org/exchange'}  # Default namespace
    
    patent_data = []  # List to store the extracted data

    # Loop through each publication reference and extract the country and doc-number
    for publication in root.findall(".//ops:publication-reference", namespaces={"ops": "http://ops.epo.org"}):
        # Get the country from each publication reference
        country = publication.find(".//document-id/country", namespaces=namespaces)
        doc_number = publication.find(".//document-id/doc-number", namespaces=namespaces)

        # Collect data if available
        if country is not None and doc_number is not None:
            patent_data.append({
                "country": country.text,
                "doc_number": doc_number.text
            })

    return patent_data
    

In [13]:
def new_search_for_patents_by_year(year, token, b, e):
    
    search_url = "https://ops.epo.org/rest-services/published-data/search"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
    }

    # Parameters: Retrieve all patents (max 100 at a time)
    params = {
        "q": f'pd within "{year},{year}"',
        "Range": f"{b}-{e}"
    }
    
    # Send request
    response = requests.get(search_url, headers=headers, params=params)

    # Check response
    if response.status_code == 200:
        xml_data = response.text
        print(xml_data)
    else:
        return 
    
#print(search_for_patents_by_year(1883, token))

In [14]:
print(new_search_for_patents_by_year(1782, token, 1, 100))

<?xml version="1.0" encoding="UTF-8"?><?xml-stylesheet type="text/xsl" href="../../style/exchange.xsl"?>
<ops:world-patent-data xmlns="http://www.epo.org/exchange" xmlns:ops="http://ops.epo.org" xmlns:xlink="http://www.w3.org/1999/xlink">
    <ops:biblio-search total-result-count="1">
        <ops:query syntax="CQL">pd within "1782,1782"</ops:query>
        <ops:range begin="1" end="100"/>
        <ops:search-result>
            <ops:publication-reference system="ops.epo.org" family-id="44501723">
                <document-id document-id-type="docdb">
                    <country>GB</country>
                    <doc-number>178201321</doc-number>
                    <kind>A</kind>
                </document-id>
            </ops:publication-reference>
        </ops:search-result>
    </ops:biblio-search>
</ops:world-patent-data>
None


In [15]:
import requests
import xml.etree.ElementTree as ET

def search_and_extract_patents(year, token, b=1, e=100, all_patents=None):
    if all_patents is None:
        all_patents = []

    search_url = "https://ops.epo.org/rest-services/published-data/search"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
    }
    
    params = {
        "q": f'pd within "{year},{year}"',
        "Range": f"{b}-{e}"
    }
    
    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Error {response.status_code} fetching patents for {year}")
        return all_patents  # Return collected patents even if there was an error

    xml_data = response.text
    print(xml_data)  # Debugging: see the raw XML response

    # Extract data
    patents = extract_patent_data(xml_data)
    all_patents.extend(patents)

    # If exactly 100 results were returned, fetch the next batch
    if len(patents) == 100:
        return search_and_extract_patents(year, token, b + 100, e + 100, all_patents)

    return all_patents  # Return final result when fewer than 100 patents are found


def extract_patent_data(xml_data):
    if not xml_data:
        return []
    
    try:
        tree = ET.ElementTree(ET.fromstring(xml_data))
        root = tree.getroot()
    except ET.ParseError:
        print("Error parsing the XML data.")
        return []

    namespaces = {'': 'http://www.epo.org/exchange'}  # Default namespace
    patent_data = []

    for publication in root.findall(".//ops:publication-reference", namespaces={"ops": "http://ops.epo.org"}):
        country = publication.find(".//document-id/country", namespaces=namespaces)
        doc_number = publication.find(".//document-id/doc-number", namespaces=namespaces)

        if country is not None and doc_number is not None:
            patent_data.append({
                "country": country.text,
                "doc_number": doc_number.text
            })

    return patent_data


In [17]:
year = 2020
all_patents_2020 = search_and_extract_patents(year, token)
print(f"Total patents found for {year}: {len(all_patents_2020)}")


<?xml version="1.0" encoding="UTF-8"?><?xml-stylesheet type="text/xsl" href="../../style/exchange.xsl"?>
<ops:world-patent-data xmlns="http://www.epo.org/exchange" xmlns:ops="http://ops.epo.org" xmlns:xlink="http://www.w3.org/1999/xlink">
    <ops:biblio-search total-result-count="10000">
        <ops:query syntax="CQL">pd within "2020,2020"</ops:query>
        <ops:range begin="1" end="100"/>
        <ops:search-result>
            <ops:publication-reference system="ops.epo.org" family-id="66287131">
                <document-id document-id-type="docdb">
                    <country>CA</country>
                    <doc-number>185831</doc-number>
                    <kind>S</kind>
                </document-id>
            </ops:publication-reference>
            <ops:publication-reference system="ops.epo.org" family-id="94970677">
                <document-id document-id-type="docdb">
                    <country>MY</country>
                    <doc-number>178937</doc-number>
      